In [ ]:
import os
from scipy.io import wavfile
import pandas as pd
import numpy as np
import scipy.fft
import matplotlib.pyplot as plt

# import my modules (helpers.py where I stored all the functions):
import helpers as hlp
import importlib
importlib.reload(hlp);

In [ ]:
# directory with the recorded files
data_source_path='/Users/joanna.luberadzka/Projects/microson_v1_data/ku_recordings/'

# directory where to store cropped recordings:
data_save_path=data_source_path

In [ ]:
# For each recording, create a sub-directory with cropped files

markers_sec=[0,32,64,96,128,160,192,224,256,288,320,352,384]
names=['plus_party_0deg','minus_party_0deg','plus_party_30deg','minus_party_30deg',
       'plus_restaurant_0deg','minus_restaurant_0deg','plus_restaurant_30deg','minus_restaurant_30deg',
       'plus_meeting_0deg','minus_meeting_0deg','plus_meeting_30deg','minus_meeting_30deg']

for filename in sorted(os.listdir(data_source_path)):
    if filename.endswith(".wav"):
        fs, data = wavfile.read(data_source_path+filename)
        # if its a mono file (measurement mic) - make a stereo, so that further code can be easily used
        if len(data.shape)<2:
            data=np.array([data,data]).T
        recording_name=os.path.splitext(filename)[0]
        recording_save_path=data_save_path+recording_name
        os.makedirs(recording_save_path,exist_ok=True)
        for j in range(0,len(markers_sec)-1):
            start_idx=markers_sec[j]*fs
            stop_idx=markers_sec[j+1]*fs
            crop_data=data[start_idx:stop_idx,:]
            crop_name=recording_name+'_'+names[j]+'.wav'
            wavfile.write(recording_save_path+'/'+crop_name,fs,crop_data)


In [ ]:
# Create a data frame with SNR estimates

scenes=['party_0deg','party_30deg','restaurant_0deg','restaurant_30deg','meeting_0deg','meeting_30deg']
hearing_aid_snr=pd.DataFrame(columns=['recording','scene','snr_left', 'snr_right','snr_both'])
for item in sorted(os.listdir(data_save_path)):
    if os.path.isdir(data_save_path+item):
        recording_name=item
        for scene in scenes:
            fs, plus = wavfile.read(data_source_path+recording_name+'/'+recording_name+'_plus_'+scene+'.wav')
            fs, minus = wavfile.read(data_source_path+recording_name+'/'+recording_name+'_minus_'+scene+'.wav')
            s=0.5*(plus+minus)
            n=0.5*(plus-minus)
            snr_left = 10 * np.log10(hlp.power(s[:,0]) / hlp.power(n[:,0]))
            snr_right = 10 * np.log10(hlp.power(s[:,1]) / hlp.power(n[:,1]))
            snr_both =10 * np.log10(hlp.power(s) / hlp.power(n))
            new_row=pd.Series({'recording':recording_name,'scene':scene,'snr_left': snr_left, 'snr_right':snr_right,'snr_both':snr_both})
            hearing_aid_snr=pd.concat([hearing_aid_snr, new_row.to_frame().T], ignore_index=True)


hearing_aid_snr.to_csv(data_save_path+'hearing_aid_snr.csv')

In [ ]:
hearing_aid_snr

In [ ]:
# Create a data frame with benefit estimates (difference in snr between bypass and enabled)
enabled=hearing_aid_snr[hearing_aid_snr["recording"].str.contains("enabled")]

bypass=hearing_aid_snr[hearing_aid_snr["recording"].str.contains("bypass")]
ku100=hearing_aid_snr[hearing_aid_snr["recording"].str.contains("001")]
ku100=ku100.append([ku100]*int(len(enabled)/6-1))

hearing_aid_benefit=pd.DataFrame(columns=['rec1','rec2','scene',
                                          'in_vs_bypass_left', 'in_vs_bypass_right',
                                          'in_vs_enabled_left','in_vs_enabled_right',
                                          'bypass_vs_enabled_left','bypass_vs_enabled_right'])

hearing_aid_benefit['rec1']=bypass['recording'].values
hearing_aid_benefit['rec2']=enabled['recording'].values
hearing_aid_benefit['scene']=enabled['scene'].values  
hearing_aid_benefit['in_vs_bypass_left']=bypass['snr_left'].values-ku100['snr_left'].values
hearing_aid_benefit['in_vs_bypass_right']=bypass['snr_right'].values-ku100['snr_right'].values
hearing_aid_benefit['in_vs_enabled_left']=enabled['snr_left'].values-ku100['snr_left'].values
hearing_aid_benefit['in_vs_enabled_right']=enabled['snr_right'].values-ku100['snr_right'].values
hearing_aid_benefit['bypass_vs_enabled_left']=enabled['snr_left'].values-bypass['snr_left'].values
hearing_aid_benefit['bypass_vs_enabled_right']=enabled['snr_right'].values-bypass['snr_right'].values


hearing_aid_benefit.to_csv(data_save_path+'hearing_aid_benefit.csv')